# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [39]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [43]:
with open ("words_250000_train.txt", 'r') as file:
  words = file.read().splitlines()

LOADING BI LSTM MODEL

In [27]:
from google.colab import files
uploaded = files.upload()

BI-LSTM Model

In [44]:
chars = sorted(list(set(''.join(words) + '0')))
char_to_int = dict((c,i) for i,c in enumerate(chars))
int_to_char = dict((i,c) for i,c in enumerate(chars))
max_word_length = 32

In [45]:
import numpy as np
from keras.layers import Conv1D, LSTM, Dense, TimeDistributed, Embedding, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Bidirectional

In [46]:
def preprocess_data(words, char_to_int, max_word_length = 32):

    X = []
    Y = []

    for word in words:
        dic = {cha : [] for cha in list(set(word))}

        for i in range(len(word)):
            dic[word[i]].append(i)

        for cha, li in dic.items():

            new_word = word
            x = []

            for ele in new_word:
                x.append(char_to_int[ele])

            for pos in li:
                x[pos] = 0
            X.append(x)
            Y.append([char_to_int[char] for char in word])

    X = pad_sequences(X,maxlen = max_word_length, padding = 'post')
    Y = pad_sequences(Y,maxlen = max_word_length, padding = 'post')
    return X,Y

In [32]:
X,Y = preprocess_data(words,char_to_int)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 42)
Y_train = to_categorical(Y_train, num_classes=len(chars))
Y_test = to_categorical(Y_test, num_classes=len(chars))

In [1]:
##Bi-LSTM Model
def create_model():
  model = Sequential()
  model.add(Embedding(input_dim=len(chars), output_dim=128, trainable=True))
  model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
  model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.001))))
  model.add(Dropout(0.4))
  model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)

In [2]:
def train_model(model):
  model.fit(X_train,Y_train, batch_size = 21, epochs = 2,validation_data = (X_test,Y_test))

In [47]:
def predict_word(model, word_with_missing, char_to_int, int_to_char, guessed_letters):
    word_encoded = []

    for char in word_with_missing:
        if(char == '0'):
            word_encoded.append(char_to_int['0'])
        else:
            word_encoded.append(char_to_int[char])

    #print(word_encoded)
    word_padded = pad_sequences([word_encoded], maxlen = 32, padding = 'post')
    #print(word_padded)

    prediction = model.predict(word_padded)

    predict_vector = prediction[0]

    for i,char in enumerate(word_with_missing):
        if(char == '0'):

            proba = predict_vector[i]
            sorted_values = np.argsort(-proba)

            for val in sorted_values:
                predicted_char = int_to_char[val]
                if(predicted_char != '0' and predicted_char not in guessed_letters):
                    return predicted_char
    return None

In [48]:
import tensorflow as tf
model = tf.keras.models.load_model('hangman_updated.h5')

In [55]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout


        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.keys = set([])
        self.n_len = 9
        self.dic = self.pre_process(self.full_dictionary)
        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
#         return None
        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def pre_process(self, dict):

      # get frequency of all sub_words upto n_len in each word.
        dic = {}
        keys = set([])

        for word in dict:
            n = len(word)
            for l in range(1, self.n_len+1):
                for i in range(0, n-l+1):
                    w = word[i:i+l]

                    if w in keys:
                        dic[w] += 1
                    else:
                        keys.add(w)
                        dic[w] = 1
        self.keys = keys
        return dic

    def get_frequency(self, word, num):

      ch = chr(ord('a') + num)

      w = ''  #new_word
      for c in word:
        if c == '_':
          c = ch
        w += c
      if w in self.keys:
        return self.dic[w]

      return 0

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################


        word = word[::2]

        ## Switch to Bi-LSTM if 70% of word is predicted.

        if word.count('_') <= 0.3*(len(word)):
            word_with_missing = word.replace('_', '0')
            return predict_word(model, word_with_missing, char_to_int, int_to_char, self.guessed_letters)

        proba = [0] * 26
        n_word = len(word)
        for n in range(1, self.n_len+1):
            if n > n_word:
                break
            for i in range(0, n_word-n+1):
                w = word[i:i+n]

                if w.count('_') != 1:
                    continue
                cnt = [0] * 26
                tot = 0
                for j in range(0, 26):
                    ch = chr(ord('a')+j)
                    if ch not in self.guessed_letters:
                            cnt[j] += self.get_frequency(w, j)
                            tot += cnt[j]
                if tot == 0:
                    continue
                for j in range(0, 26):
                    cnt[j] = cnt[j]/tot
                    proba[j] += cnt[j] * n  ## weight = n


        maxi = 0
        val = 0

        for i in range(26):
          ch = chr(ord('a')+i)
          if ch not in self.guessed_letters:
            if proba[i] > maxi:
              maxi = proba[i]
              val = i

        res = chr(ord('a') + val)
        return res


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = set([])
        self.current_dictionary = self.full_dictionary
#         return None
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.add(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [56]:
api = HangmanAPI(access_token="2d169b2b41bd53d6055dccd4510ea6", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [58]:
for i in range(200):
  api.start_game(practice=1,verbose=True)
  [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
  practice_success_rate = total_practice_successes / total_practice_runs
  print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Streaming output truncated to the last 5000 lines.
Guessing letter: i
Sever response: {'game_id': 'ade604836669', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ i _ _ '}
Guessing letter: n
Sever response: {'game_id': 'ade604836669', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ n _ _ i n _ '}
Guessing letter: g
Sever response: {'game_id': 'ade604836669', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ n _ _ i n _ '}
Guessing letter: t
Sever response: {'game_id': 'ade604836669', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ n _ _ i n t '}
Guessing letter: a
Sever response: {'game_id': 'ade604836669', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ n _ _ i n t '}
Guessing letter: o
Sever response: {'game_id': 'ade604836669', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o n _ o i n t '}
Guessing letter: j
Sever response: {'game_id': 'ade604836669', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o n j o i n t '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [59]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Playing  1  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Playing  2  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Playing  3  th game
Playing  4  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Playing  5  th game
Playing  6  th game
Playing  7  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Playing  8  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Playing  9  th game
Playing  10  th game
Playing  11  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Playing  12  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Playing  13  th game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Playing  14  th game
1/1 ━━━━━━━

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)